\begin{equation}
W(r)=k \begin{cases}1-6\left(\frac{r}{h}\right)^{2}+6\left(\frac{r}{h}\right)^{3} & \text { for } 0 \leq \frac{r}{h}<1 / 2 \\ 2\left(1-\frac{r}{h}\right)^{3} & \text { for } 1 / 2 \leq \frac{r}{h}<1 \\ 0 & \text { otherwise }\end{cases}
\end{equation}

### (a)  
\begin{equation*}
    \int d \bar r \, W(r) = 2 \pi \int dr \, rW(r)
\end{equation*}
Let evaluate each part separately using the change of variable $r \to x = r/h$ (omitting k) 
\begin{gather*}
    2 \pi h^2 \int_{0}^{1/2} x (1 - 6 x^2 + 6 x^3) \, dx = 2\pi h^2 \left[ \frac{x^2}{2} - \frac{3}{2} x^2 + \frac{6}{5} x^5 \right]_0^{1/2} = 2\pi h^2 \left( \frac{1}{8} - \frac{3}{32} + \frac{3}{80} \right) = \frac{11}{80} \pi h^2
\end{gather*}
\begin{gather*}
    2\pi h^2 \int_{1/2}^1 2x \left( 1 - x \right)^3 \, dx = 4 \pi h^2 \int_{1/2}^1 x - x^4 - 3x^2 + 3x^3 \, dx = 4\pi h^2 \left[ \frac{x^2}{2} - \frac{x^5}{5} - x^3 + \frac{3}{4}x^4 \right]_{1/2}^{1} = \frac{\pi}{2} h^2 \left[ 3 - \frac{31}{120} - 7 + \frac{45}{8} \right] = \frac{3}{80} \pi h^2
\end{gather*}
Hence
\begin{equation*}
    \int d \bar r \, W(r) = k \left(\frac{11}{80} + \frac{3}{80} \right) \pi h^2 = \frac{7}{40} \pi h^2 k \qquad \Longrightarrow \qquad k = \frac{40}{7\pi h^2}
\end{equation*}

### (b) and (c)

In [40]:
import numpy as np
import matplotlib.pyplot as plt


# Kernel function
def kernel_func(r, h):
        k = 40/(7*np.pi*h**2)
        r = abs(r/h)
        if r >= 1:
            return 0
        elif r < 1/2:
            return k * (1 - 6*r**2 + 6*r**3)
        else:
            return 2*k * (1 - r)**3

#Reads a square image in 8-bit/color PPM format from the given file. Note: No checks on valid format are done.
def readImage(filename):
    f = open(filename,"rb")
    
    f.readline()
    s = f.readline()
    f.readline()
    (pixel, pixel) = [t(s) for t,s in zip((int,int),s.split())]
    
    data = np.fromfile(f,dtype=np.uint8,count = pixel*pixel*3)
    img = data.reshape((pixel,pixel,3)).astype(np.double)
    
    f.close()
    
    return img, pixel
    

#Writes a square image in 8-bit/color PPM format.
def writeImage(filename, image):
    f = open(filename,"wb")
    
    pixel = image.shape[0]
    ppm_header = f'P6 ' + str(pixel) + ' ' + str(pixel) + ' ' + str(255) +'\n'
    f.write(bytearray(ppm_header,'utf-8'))
    
    image = image.astype(np.uint8)
    
    image.tofile(f)
    
    f.close() 
    
    
    

img, pixel = readImage("./FoSM_Tutorial_06/aq-original.ppm")

sums_before = [np.sum(img[:,:,i], (0,1)) for i in range(3)]
print("Red before:", sums_before[0])
print("Green before:", sums_before[1])
print("Blue before:", sums_before[2])


#Now we set up our desired smoothing kernel. We'll use complex number for it even though it is real. 
kernel_real = np.zeros((pixel,pixel),dtype=np.complex)

hsml = 10.

#now set the values of the kernel 
for i in np.arange(pixel):
    for j in np.arange(pixel):
        #TODO: do something sensible here to set the real part of the kernel
        #kernel_real[i, j] = ....
        r = np.sqrt((i-pixel/2)**2 + (j-pixel/2)**2)
        kernel_real[i,j] = kernel_func(r, hsml)


#Let's calculate the Fourier transform of the kernel
kernel_kspace = np.fft.fft2(kernel_real)


#further space allocations for image transforms
color_real = np.zeros((pixel,pixel),dtype=np.complex)


#we now convolve each color channel with the kernel using FFTs
for colindex in np.arange(3):
    #copy input color into complex array
    color_real[:,:].real = img[:,:,colindex]
    
    
    #forward transform
    color_kspace = np.fft.fft2(color_real)
    
    #multiply with kernel in Fourier space
    #TODO: fill in code here
    color_kspace = color_kspace * kernel_kspace
    
    #backward transform
    color_real = np.fft.ifft2(color_kspace)

    #copy real value of complex result back into color array
    img[:,:,colindex] = color_real.real

sums_after = [np.sum(img[:,:,i], (0,1)) for i in range(3)]

print("Red after:", sums_before[0])
print("Green after:", sums_before[1])
print("Blue after:", sums_before[2])

print("Red relative variation:", abs(sums_after[0].real-sums_before[0])/sums_before[0]*100, "%")
print("Green relative variation:", abs(sums_after[1].real-sums_before[1])/sums_before[1]*100, "%")
print("Blue relative variation:", abs(sums_after[2].real-sums_before[2])/sums_before[2]*100, "%")
    
writeImage("aq-smoothed.ppm", img)


Red before: 25182443.0
Green before: 12287845.0
Blue before: 24795222.0


/tmp/ipykernel_15745/2088666872.py:59: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kernel_real = np.zeros((pixel,pixel),dtype=np.complex)


[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
Red after: 25182443.0
Green after: 12287845.0
Blue after: 24795222.0
Red relative variation: 0.00038556912996194235 %
Green relative variation: 0.0003855691299473949 %
Blue relative variation: 0.0003855691299

/tmp/ipykernel_15745/2088666872.py:77: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  color_real = np.zeros((pixel,pixel),dtype=np.complex)
